<a href="https://www.kaggle.com/vidushibhatia/nlp-on-tweets?scriptVersionId=84709362" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
sample = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [4]:
########################################################################################################

def TextClassifier(text_data,label):
    text_clf = Pipeline([ ('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB())])
    text_clf = text_clf.fit(text_data, label)
    return text_clf


########################################################################################################

def TextClassificationCompile(data):
    text_classification = data[pd.notnull(data['text'])]
    text_classification = text_classification[['id','text','target']]
    clf_model_target = TextClassifier(text_classification.text, text_classification.target)

    return clf_model_target

########################################################################################################


In [5]:
# Text Classification Train
y = train['target']
clf_model_target = TextClassificationCompile(train)
text_classification = train[pd.notnull(train['text'])]
text_classification['target_classification'] = clf_model_target.predict(text_classification.text)
text_classification = text_classification[['id', 'target_classification']]
train = train.merge(text_classification, how='left', on='id')
train = train.drop(['target'],axis = 1)
train['flag'] = 1

# Test Prediction Train
test['target_classification'] = clf_model_target.predict(test.text)
test['flag']=0

full_data = pd.concat([train, test])
full_data

,id,keyword,location,text,target_classification,flag
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,0,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,0,1
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1,0
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,0,0
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1,0
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1,0


In [6]:
# One hot encoding on full data
tab = pd.get_dummies(full_data['keyword']).reset_index()
full_data = full_data.reset_index().rename(columns={full_data.index.name:'index'})
tab = pd.get_dummies(full_data['keyword']).reset_index().rename(columns={tab.index.name:'index'})
full_data = full_data.merge(tab, how="inner",on="index")
tab = pd.get_dummies(full_data['location']).reset_index().rename(columns={tab.index.name:'index'})
full_data = full_data.merge(tab, how="inner",on="index")
full_data

,index,id,keyword,location,text,target_classification,flag,ablaze,accident,aftershock,...,"ÌÏT: 6.4682,3.18287","ÌÏT: 6.488400524109015,3.352798039832285","ÌÏT: 7.384559,3.8793718","ÌøåÀå_T: 40.736324,-73.990062",å_: ?? ÌÑ ? : ?,å_å_Los Mina Cityã¢,å¡å¡Midwest Û¢Û¢,å©hicago,åÊ(?Û¢`?Û¢å«)??,åø\_(?)_/åø
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,NaN,NaN,Just happened a terrible car crash,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10871,7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10872,7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10873,7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10874,7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
col_list=[]
i=0
for col in full_data.columns:
    col_list.append('var'+str(i))
    i=i+1    
full_data.columns = col_list

full_data

,var0,var1,var2,var3,var4,var5,var6,var7,var8,var9,...,var4739,var4740,var4741,var4742,var4743,var4744,var4745,var4746,var4747,var4748
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,NaN,NaN,Just happened a terrible car crash,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10871,7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10872,7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10873,7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10874,7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = full_data[full_data['var6']==1].drop(['var0','var1','var2','var3','var4','var6'], axis=1) 
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=123)
ID = full_data[full_data['var6']==0]['var1']
test_X = full_data[full_data['var6']==0].drop(['var0','var1','var2','var3','var4','var6'], axis=1) 

In [9]:
############################################ Helper Function 8 ############################################################
def runXGB(X, y, X_val, y_val, hyperparams):
    model = XGBClassifier(
        learning_rate = hyperparams['lr'],
        n_estimators = hyperparams['n'],
        max_depth = hyperparams['depth'],
        min_child_weight = hyperparams['child_weight'],
        scale_pos_weight = hyperparams['scale_pos'],
        gamma=hyperparams['gamma'],
        subsample=hyperparams['sub-sample'],
        colsample_bytree=hyperparams['co-sample'],
        objective= 'binary:logistic',
#         scoring = 'f1',
        nthread=4,
        seed=27)

    model.fit(X, y, eval_metric=["error", "logloss"],eval_set = [(X,y), (X_val, y_val)] ,verbose = False)
#     model.fit(X, y,verbose = False)
    # F1 score
    predictions = model.predict(X)
    score_1 = f1_score( y,  predictions, average='weighted')
    print("F1 Score for Train Set: %.2f%%" % (score_1 * 100.0))
    predictions = model.predict(X_val)
    score_1 = f1_score( y_val,  predictions, average='weighted')
    print("F1 Score for Validation Set: %.2f%%" % (score_1 * 100.0))
    return model

In [10]:
from sklearn.metrics import classification_report,confusion_matrix
hyperparams = {'lr': 0.07, 'n':100, 'depth': 5, 'child_weight':1, 
               'scale_pos':1, 'sub-sample':0.8, 'co-sample':0.8, 'gamma': 0}

# model_1  = runXGB(train_X, pd.to_numeric(train_y), valid_X, pd.to_numeric(valid_y),hyperparams)
model_1  = runXGB(train_X, train_y, valid_X, valid_y, hyperparams)

F1 Score for Train Set: 88.49%
F1 Score for Validation Set: 87.75%


In [11]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('full_data', 55099898),
 ('tab', 49257564),
 ('X', 36222686),
 ('train_X', 28976252),
 ('test_X', 15525386),
 ('valid_X', 7246466),
 ('train', 2404370),
 ('test', 1008584),
 ('text_classification', 182744),
 ('train_y', 97472),
 ('y', 61064),
 ('sample', 52368),
 ('ID', 52240),
 ('col_list', 38224),
 ('valid_y', 24400),
 ('XGBClassifier', 2008),
 ('CountVectorizer', 1472),
 ('MultinomialNB', 1192),
 ('Pipeline', 1064),
 ('TfidfTransformer', 1064),
 ('hyperparams', 376),
 ('TextClassificationCompile', 144),
 ('TextClassifier', 144),
 ('classification_report', 144),
 ('confusion_matrix', 144),
 ('f1_score', 144),
 ('runXGB', 144),
 ('train_test_split', 144),
 ('filenames', 104),
 ('np', 88),
 ('pd', 88),
 ('plt', 88),
 ('sns', 88),
 ('stats', 88),
 ('col', 84),
 ('dirname', 82),
 ('clf_model_target', 64),
 ('model_1', 64),
 ('filename', 57),
 ('i', 28)]

In [12]:
del train, train_X, train_y, valid_X, valid_y, full_data, X, y, col_list, XGBClassifier,text_classification, tab

In [13]:
test_X['prediction'] = model_1.predict(test_X)

In [14]:
test_X = test_X.merge(ID, left_index=True, right_index=True)
test_X

,var5,var7,var8,var9,var10,var11,var12,var13,var14,var15,...,var4741,var4742,var4743,var4744,var4745,var4746,var4747,var4748,prediction,var1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,10861
6519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10865
6521,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,10868
6523,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,10874


In [15]:
my_submission = test_X [['var1','prediction']].rename(columns={'var1':'id', 'prediction':'target'})
my_submission.to_csv('/kaggle/working/submission.csv', index=False)